## Notebook to adjust PlanetScope images using the estimated surface reflectance of snow

Rainey Aberle

Summer 2022

__To-Do:__
- option to remove intermediate files (i.e., if hillshade correction and radiometric adjustment are selected, remove hs-corrected file)
- option to filter images with clipped bands

In [ ]:
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import rasterio as rio
from shapely.geometry import Polygon
from scipy.interpolate import interp2d
import time
import rasterio as rio
import geopandas as gpd
import sys

### 1. Define paths in directory, snow-covered area (SCA), and settings

In [ ]:
# -----Define paths in directory
# path to planet-snow/ directory
base_path = '/Users/raineyaberle/Research/PhD/Planet_snow_cover/planet-snow/'
# path to raw PlanetScope images
im_path = base_path+'../study-sites/SCascade_LeConte/imagery/Planet/stitched/'
# # path to hillshades directory
# hs_path = base_path+'../study-sites/Wolverine/imagery/Planet/hillshades/'
# output folder
out_path = im_path+'../adjusted-filtered/'

# -----Define snow-covered area (SCA) 
# Wolverine
# xmin, xmax, ymin, ymax = 393.5e3, 395.5e3, 6699.8e3, 6701e3
# SitKusa
# xmin, xmax, ymin, ymax = 557e3, 560e3, 6668e3, 6670e3
# Mendenhall
# xmin, xmax, ymin, ymax = 533.2e3, 535.2e3, 6490e3, 6492e3
# Easton
# xmin, xmax, ymin, ymax = 585.6e3, 586.1e3, 5400.6e3, 5401e3
# Blue
# xmin, xmax, ymin, ymax = 447e3, 447.5e3, 5294.1e3, 5294.5e3
# Emmons
# xmin, xmax, ymin, ymax = 595e3, 595.9e3, 5189.1e3, 5190.1e3
# SCascade_LeConte
xmin, xmax, ymin, ymax = 644.1e3, 644.6e3, 5356.6e3, 5356.9e3

SCA = Polygon([(xmin, ymin), (xmax, ymin), (xmax, ymax), (xmin, ymax), (xmin, ymin)])

# -----Determine settings
plot_results = True # = True to plot corrected images
hillshade_correction = False # = True to apply hillshade correction using a DEM
skip_clipped = False # = True to skip images where bands appear "clipped", i.e. max blue SR < 0.8
adjust_radiometry = True # = True to adjust image band radiometry using the SCA
# path to DEM - required only if hillshade_correction = True
# DEM_path = base_path+'../study-sites/Wolverine/DEMs/WG_20200728-DEM_mosaic_crop_UTM6_resamp_filled.tif'

# -----Add path to functions
sys.path.insert(1, base_path+'functions/')
from image_adjustment_utils import sunpos, into_range, apply_hillshade_correction, adjust_image_radiometry

### 2. Load DEM and PlanetScope images

In [ ]:
# -----Load DEM
if hillshade_correction:
    DEM = rio.open(DEM_path)
    # coordinates
    DEM_x = np.linspace(DEM.bounds.left, DEM.bounds.right, num=np.shape(DEM.read(1))[1])
    DEM_y = np.linspace(DEM.bounds.top, DEM.bounds.bottom, num=np.shape(DEM.read(1))[0])
    print('DEM CRS: ',DEM.crs) # print crs

# -----Load images
ims = os.chdir(im_path) # change directory
im_names = glob.glob('*.tif') # load all SR files
im_names.sort() # sort file names by date

# -----Read one image and plot SCA
im_name = im_names[2]
im = rio.open(im_name)
print('Images CRS:',im.crs) # print crs
im_scalar = 10000 # scalar multiplier for SR values
b = im.read(1).astype(float)
g = im.read(2).astype(float)
r = im.read(3).astype(float)
nir = im.read(4).astype(float)
# divide by im_scalar if they have not been already
if (np.nanmax(b)>1e3):
    b = b / im_scalar
    g = g / im_scalar
    r = r / im_scalar
    nir = nir / im_scalar
# define coordinates grid
im_x = np.linspace(im.bounds.left, im.bounds.right, num=np.shape(b)[1])
im_y = np.linspace(im.bounds.top, im.bounds.bottom, num=np.shape(b)[0])
# plot
fig, ax1 = plt.subplots(1, 1, figsize=(8,8))
plt.rcParams.update({'font.size': 14, 'font.serif': 'Arial'})
ax1.imshow(np.dstack([r, g, b]), 
           extent=(np.min(im_x)/1000, np.max(im_x)/1000, np.min(im_y)/1000, np.max(im_y)/1000))
ax1.plot([x/1000 for x in SCA.exterior.xy[0]], [y/1000 for y in SCA.exterior.xy[1]], 
         color='black', linewidth=2, label='SCA')
ax1.set_xlabel('Easting [km]')
ax1.set_ylabel('Northing [km]')
ax1.set_title(im_name[0:15])
ax1.legend()
plt.show()

### 3. Loop through images

In [ ]:
# -----Start timer
t1 = time.monotonic()

# -----Loop through images
for im_name in im_names:
    
    # load image
    print('Image: ',im_name)
    im = rio.open(im_name)
    
    # apply hillshade correction
    if (hillshade_correction == True):
        im_corrected_name = apply_hillshade_correction(CRS, SCA, im, im_name, im_path, DEM_path, hs_path, out_path, skip_clipped, plot_results)
        if im_corrected_name=='N/A':
            print('----------')
            print(' ')            
            continue
        else:
            im_corrected = rio.open(out_path+im_corrected_name)
    
    # adjust radiometry
    if (adjust_radiometry == True):
        if (hillshade_correction==True):
            im_adj_name = adjust_image_radiometry(im_corrected, im_corrected_name, out_path, SCA, out_path, skip_clipped, plot_results)
        else:
            im_adj_name = adjust_image_radiometry(im, im_name, im_path, SCA, out_path, skip_clipped, plot_results)
    
    print('----------')
    print(' ')

# -----Stop timer
t2 = time.monotonic()
print('Time elapsed: ',str(round(t2-t1)/60),'minutes ')
    